In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

: 

In [ ]:
# opening words
words = open('names.txt', 'r').read().splitlines()
words[:8]

: 

In [ ]:
# build vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos  = {i:s for s,i in stoi.items()}
print(itos)

: 

In [ ]:
# build dataset
block_size = 3 # context length, how many characters taken to predict next character
X, Y = [], [] # input, label for nn
for w in words:
    # print(w)
    context = [0] * block_size # padding
    for ch in w + '.':
        ix = stoi[ch] # finds the corresponding number, adds to array Y
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)

: 

In [ ]:
X.shape, X.dtype, Y.shape, Y.dtype

: 

Cleaned version

In [ ]:
import random

def build_dataset(words):
    block_size = 3
    X, Y = [], []
    
    for w in words:
        context = [0] * block_size # padding
        for ch in w + '.':
            ix = stoi[ch] # finds the corresponding number, adds to array Y
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

: 

In [ ]:
Xtr.shape, Ytr.shape # dataset

: 

In [ ]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]


: 

In [ ]:
sum(p.nelement() for p in parameters) # total number of parameters

: 

In [ ]:
for p in parameters:
    p.requires_grad = True

: 

In [ ]:
lre = torch.linspace(-3, 0, 1000) # learning rate exponential
lrs = 10**lre
# creates random learning rates between 0.001 and 1

: 

In [ ]:
lri = []
lossi = []
stepi = []

: 

In [ ]:
for i in range(200000):
    # for i in range(2000000):

    # construct minibatch
    ix = torch.randint(0, Xtr.shape[0], (32,))

    # forward pass
    emb = C[Xtr][ix] # embedded lookup table
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1) # first layer
    logits = h @ W2 + b2 # second layer
    # counts = logits.exp()
    # prob = counts / counts.sum(1, keepdims=True)
    # loss = -prob[torch.arange(32, Y)].log().mean()
    # useless, inefficient, only educational 
    loss = F.cross_entropy(logits, Ytr[ix])

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # updating
    lr = 0.1
    # lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    # stats
    stepi.append(i)
    lossi.append(loss.log10().item())

: 

In [ ]:
plt.plot(stepi, lossi)

: 

In [ ]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss

: 

In [ ]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss

: 

In [ ]:
# training split, validation split, test split
# 8:1:1

: 

In [ ]:
plt.figure(figsize=(8,8))
plt.scatter(C[:, 0].data, C[:, 1].data, s=200)
for i in range(C.shape[0]):
    plt.text(C[i, 0].item(), C[i, 1].item(), itos[i], ha="center", va="center", color='white')
plt.grid('minor')

: 

In [ ]:
# sampling from model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(100):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    
    print(''.join(itos[i] for i in out))

: 